In [1]:
%pip install torch transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a Hugging Face course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
print(input_ids)

# This line fails because the input ids are for a single sequence instead of a list of sequences
model(input_ids)

tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,  2227,  2607,
         2026,  2878,  2166,  1012])


IndexError: too many indices for tensor of dimension 1

In [8]:
tokenized_inputs = tokenizer(sequence, return_tensors='pt')
print(tokenized_inputs['input_ids'])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,  2227,
          2607,  2026,  2878,  2166,  1012,   102]])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print(f'Input IDs: {input_ids}')

output = model(input_ids)
print(f'Logits: {output.logits}')

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [10]:
batched_ids = [ids, ids]
input_ids = torch.tensor(batched_ids)
print(f'Input IDs: {input_ids}')

output = model(input_ids)
print(f'Logits: {output.logits}')

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [11]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

In [12]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id]
]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


In [16]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0]
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [20]:
sequence_1 = 'I have been waiting for a Hugging Face course my whole life.'
sequence_2 = 'I hate this so much!'

tokens_1 = tokenizer.tokenize(sequence_1)
tokens_2 = tokenizer.tokenize(sequence_2)

ids_1 = tokenizer.convert_tokens_to_ids(tokens_1)
ids_2 = tokenizer.convert_tokens_to_ids(tokens_2)
print(ids_1)
print(ids_2)

[1045, 2031, 2042, 3403, 2005, 1037, 17662, 2227, 2607, 2026, 2878, 2166, 1012]
[1045, 5223, 2023, 2061, 2172, 999]


In [31]:
batched_ids = [
    [tokenizer.cls_token_id, 1045, 2031, 2042, 3403, 2005, 1037, 17662, 2227, 2607, 2026, 2878, 2166, 1012, tokenizer.sep_token_id],
    [tokenizer.cls_token_id, 1045, 5223, 2023, 2061, 2172, 999, tokenizer.sep_token_id] + [tokenizer.pad_token_id] * 7
]

attention_mask = [
    [1] * 15,
    [1] * 8 + [0] * 7
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[-3.0584,  3.2022],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [30]:
inputs = tokenizer([sequence_1, sequence_2], padding=True, truncation=True, return_tensors='pt')
print(inputs)
outputs = model(**inputs)
print(outputs.logits)

{'input_ids': tensor([[  101,  1045,  2031,  2042,  3403,  2005,  1037, 17662,  2227,  2607,
          2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}
tensor([[-3.0584,  3.2022],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


Truncate the sequence to avoid crashing the model if the sequence is longer than the model can handle

In [ ]:
sequence = sequence[:max_sequence_length]